## 4.5. 读取和存储
到目前为止，我们介绍了如何处理数据以及如何构建、训练和测试深度学习模型。然而在实际中，我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用。

In [1]:
import tensorflow as tf
import numpy as np

### 4.5.1. 读写`Tensor`

我们可以直接使用`save`函数和`load`函数分别存储和读取。下面的例子创建了tensor`x`，并将其存在文件名同为x的文件里。Tensorflow暂时没有保存和读取矩阵数据，只能保存和读取模型。

In [2]:
x = tf.ones(3)
np.save('x.npy',x)

然后我们将数据从存储的文件读回内存。

In [3]:
x2 = np.load('x.npy')
x2

array([1., 1., 1.], dtype=float32)

我们还可以存储一列`tensor`并读回内存。

In [4]:
y = tf.ones(4)
np.save('xy.npy',[x,y])
x2,y2 = np.load('xy.npy',allow_pickle=True)
(x2,y2)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>)

我们甚至可以存储并读取一个从字符串映射到`tensor`的字典。

In [5]:
mydict = {'x':x,'y':y}
np.save('mydict.npy',mydict)
mydict2 = np.load('mydict.npy',allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>},
      dtype=object)

### 4.5.2. 读写Keras模型的参数
`Keras`的`Model`类提供了`get_weights()`函数和`save_weights`函数来读写模型参数。为了演示方便，我们先创建一个多层感知机，并将其初始化。

In [6]:
import tensorflow.keras as keras

class MLP(keras.Model):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.flatten = keras.layers.Flatten()
        self.hidden = keras.layers.Dense(256, activation='relu')
        self.outputs = keras.layers.Dense(10)

    def call(self, x):
        inputs = self.flatten(x)
        hid = self.hidden(inputs)
        out = self.outputs(hid)
        return out

net = MLP()
X = tf.random.uniform(shape=(2, 20))
Y = net(X)
X,Y

(<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
 array([[0.72066367, 0.74439263, 0.8329483 , 0.877746  , 0.7424873 ,
         0.70886314, 0.40852106, 0.46363497, 0.8317559 , 0.612955  ,
         0.7960923 , 0.12308407, 0.24153852, 0.5242946 , 0.9429413 ,
         0.1000005 , 0.27063715, 0.6172676 , 0.20213592, 0.8636596 ],
        [0.4910003 , 0.5663651 , 0.36159134, 0.73424375, 0.76822996,
         0.81424034, 0.9148189 , 0.23447478, 0.9114976 , 0.10812128,
         0.2592802 , 0.91107   , 0.3662306 , 0.31527257, 0.12566042,
         0.17648566, 0.16609132, 0.7959317 , 0.26270187, 0.10316133]],
       dtype=float32)>,
 <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
 array([[ 0.12259766, -0.16459455, -0.22024137, -0.29355842,  0.06675099,
         -0.28008884, -0.20612115, -0.0884061 , -0.43741766,  0.21470258],
        [-0.06200828, -0.20888153, -0.21636906, -0.2486872 , -0.10419051,
         -0.06668977, -0.07280136, -0.234931  , -0.21029462, -0.1390564 ]],
       dtype=float32)>

下面把该模型的参数存成文件，文件名为`mlp_params.h5`。[参考官方文档](https://tensorflow.google.cn/guide/keras/save_and_serialize#part_ii_saving_and_loading_of_subclassed_models)

In [7]:
filename = 'mlp_params.h5'
net.save_weights(filename)

接下来，我们再实例化一次定义好的多层感知机。与随机初始化模型参数不同，我们在这里直接读取保存在文件里的参数。

因为这两个实例都有同样的模型参数，那么对同一个输入X的计算结果将会是一样的。我们来验证一下。

In [8]:
net2 = MLP()
net2(X)
net2.load_weights(filename)
Y2 = net2(X)
Y2 == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>